In [118]:
import pandas as pd
import matplotlib.pyplot as plt

In [119]:
complete_df = pd.read_excel('data_okeanos.xlsx')

In [120]:
def opschonen(data):
    ### Onnodige kolommen verwijderen
    complete_df = pd.read_excel(data)
    clean_col_df = complete_df.drop(columns=['datum', 'ploeg', 'zone', 'intervaltype', 'interval_afstand', 'aantal_intervallen', 'interval_tijd', 'interval_nummer', 'rust', 'machine', 'spm'], errors='ignore')

    ### Lege strings omzetten naar NaN in de specifieke kolommen
    clean_col_df['500_split'] = clean_col_df['500_split'].replace('', pd.NA)
    clean_col_df['2k tijd'] = clean_col_df['2k tijd'].replace('', pd.NA)

    ### Alleen rijen met lege '500_split' of '2k tijd' verwijderen
    clean_row_df = clean_col_df.dropna(subset=['500_split', '2k tijd'])
    clean_row_df.dropna(how='any',inplace=True)

    ### Maak een kopie voor opschonen
    clean_df = clean_row_df.copy()
    clean_df.dropna(subset=['500_split', '2k tijd'])

    ### Komma's naar punten veranderen in ‘500_split’ en ‘2k tijd’ columns en uren veranderen naar minuten (00:01:11.11 wordt 1:11.11)
    for col in ['500_split', '2k tijd']:
        clean_df[col] = clean_df[col].astype(str).str.strip()
        clean_df[col] = clean_df[col].str.replace(',', '.', regex=False)
        clean_df[col] = clean_df[col].str.lstrip('0')

        # clean_df[col] = clean_df[col].astype(str).str.strip()
        # clean_df[col] = clean_df[col].str.replace(',', '.', regex=False)
        # clean_df[col] = clean_df[col].str.lstrip('0')
        # clean_df[col] = clean_df[col].str.replace('^0:0', '', regex=True)

    ### Verander 'trainingype' naar 'trainingtype'
    clean_df = clean_df.rename(columns={'trainingype': 'trainingtype'})
 
    return clean_df


### de minuten veranderen naar seconden

def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    try:
        minutes, seconds = map(float, str(time_str).split(':'))
        # print('GOOD:', time_str)
        return minutes * 60 + seconds
    except ValueError:
        # print('ERROR:', time_str)
        return None

def verander_omzetting_seconden(docu):
    docu['500_split'] = docu['500_split'].apply(time_to_seconds)
    docu['2k tijd'] = docu['2k tijd'].apply(time_to_seconds)
    return docu


In [121]:
roeien_opgeschoond = opschonen('data_okeanos.xlsx')
print(roeien_opgeschoond)

      ervaring geslacht gewichtsklasse   naam trainingtype      500_split  \
0          1.0        M              Z    z47         5x5'         1:44.6   
1          1.0        M              Z    z47         5x5'         1:44.7   
2          1.0        M              Z    z47         5x5'         1:44.3   
3          1.0        M              Z    z47         5x5'         1:44.0   
4          1.0        M              Z    z47         5x5'         1:44.1   
...        ...      ...            ...    ...          ...            ...   
7433       1.0        M              L  Z2522       1x2000  :06:37.400000   
7434       1.0        M              L  Z2521       1x2000  :06:57.500000   
7435       1.0        M              L  Z2520       1x2000         :07:05   
7436       1.0        M              Z  Z2517       1x2000         6:39.1   
7437       1.0        M              Z  Z2519       1x2000         6:17.0   

      2k tijd   2k datum  
0      6:19.9 2018-12-08  
1      6:19.9 2018-12

/tmp/ipykernel_8539/3565298680.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_row_df.dropna(how='any',inplace=True)


In [122]:
dataframe_op_sec = verander_omzetting_seconden(roeien_opgeschoond)
dataframe_op_sec.dropna(inplace=True)
print(dataframe_op_sec)

# dataframe_op_sec.to_csv('verschoond_dataframe.csv', index=False)
# verschoond_dataframe = pd.read_csv('verschoond_dataframe.csv')
# verschoond_dataframe.to_excel('verschoond_dataframe.xlsx', index=None, header=True)

      ervaring geslacht gewichtsklasse   naam trainingtype  500_split  \
0          1.0        M              Z    z47         5x5'      104.6   
1          1.0        M              Z    z47         5x5'      104.7   
2          1.0        M              Z    z47         5x5'      104.3   
3          1.0        M              Z    z47         5x5'      104.0   
4          1.0        M              Z    z47         5x5'      104.1   
...        ...      ...            ...    ...          ...        ...   
7212       0.0        V              L  D2511       1x1500      120.4   
7419       1.0        M              Z  Z2522       2x2000      387.9   
7420       1.0        M              Z  Z2522       2x2000      406.4   
7436       1.0        M              Z  Z2517       1x2000      399.1   
7437       1.0        M              Z  Z2519       1x2000      377.0   

      2k tijd   2k datum  
0       379.9 2018-12-08  
1       379.9 2018-12-08  
2       379.9 2018-12-08  
3       379.9 2

In [123]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

def omzetten_binair(df):

    ### de trainingtypes zo veranderen dat ze makkelijker te gebruiken zijn
    df['trainingtype'] = df['trainingtype'].str.replace(r"/[a-zA-Z0-9']{3,}", "", regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace("HOP+3x1'r", "57'", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace(" ", "", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace("r", "", regex=False)
    df['trainingtype'] = df['trainingtype'].str.replace(r"minuutjes", "1'", regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace(r'\b(?:\d+x)+1\'', "1'", regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace(r'(\d+x\d+)(?!m|\')$', r'\1m', regex=True)
    df['trainingtype'] = df['trainingtype'].str.replace(r'\b(\d{4})(?!m|\')\b', r'\1m', regex=True)

    ### zet de trainingtypes om naar binaire getallen
    unique_trainingtypes = df['trainingtype'].unique()
    trainingtype_to_decimal = {trainingtype: i for i, trainingtype in enumerate(unique_trainingtypes)}
    trainingtype_mapping = {trainingtype: bin(i)[2:] for trainingtype, i in trainingtype_to_decimal.items()}
    df['binary_trainingtype'] = df['trainingtype'].map(trainingtype_mapping)

    ### zet de geslachten om naar binaire getallen
    geslacht_binarizer = LabelBinarizer()
    geslacht_binarizer.fit(df['geslacht'])
    geslacht_mapping = {str(label): bin(i)[2:] for i, label in enumerate(geslacht_binarizer.classes_)}
    df['binary_geslacht'] = df['geslacht'].map(geslacht_mapping)

    ### zet de gewichtsklasses om naar binaire getallen
    gewichtsklasse_binarizer = LabelBinarizer()
    gewichtsklasse_binarizer.fit(df['gewichtsklasse'])
    gewichtsklasse_mapping = {str(label): bin(i)[2:] for i, label in enumerate(gewichtsklasse_binarizer.classes_)}
    df['binary_gewichtsklasse'] = df['gewichtsklasse'].map(gewichtsklasse_mapping)

    df['ervaring'] = df['ervaring'].astype(int)

    return df, trainingtype_mapping, geslacht_mapping, gewichtsklasse_mapping

new_dataframe, trainingtype_mapping, geslacht_mapping, gewichtsklasse_mapping = omzetten_binair(dataframe_op_sec)

print(new_dataframe.head())
print("Trainingtype Mapping:", trainingtype_mapping)
print("Geslacht Mapping:", geslacht_mapping)
print("Gewichtsklasse Mapping:", gewichtsklasse_mapping)

   ervaring geslacht gewichtsklasse naam trainingtype  500_split  2k tijd  \
0         1        M              Z  z47         5x5'      104.6    379.9   
1         1        M              Z  z47         5x5'      104.7    379.9   
2         1        M              Z  z47         5x5'      104.3    379.9   
3         1        M              Z  z47         5x5'      104.0    379.9   
4         1        M              Z  z47         5x5'      104.1    379.9   

    2k datum binary_trainingtype binary_geslacht binary_gewichtsklasse  
0 2018-12-08                   0               0                     1  
1 2018-12-08                   0               0                     1  
2 2018-12-08                   0               0                     1  
3 2018-12-08                   0               0                     1  
4 2018-12-08                   0               0                     1  
Trainingtype Mapping: {"5x5'": '0', "3x15'": '1', "3x20'": '10', '3x2000m': '11', '6000m': '100', '

In [124]:
def tweek_naar_split(data):
    mask = data['500_split'] > 300
    data.loc[mask, '500_split'] = data.loc[mask, '500_split'] / 4
    return data

In [125]:
def remove_outliers(data):
  alles = tweek_naar_split(data)
  alles = alles[alles['500_split'] >= 80]
  return alles

def total_clean(data):
    
    columns_to_drop = ['geslacht', 'gewichtsklasse', 'trainingtype', 'naam', '2k datum']
    cleaned_data = data.drop(columns=columns_to_drop)
    
    return cleaned_data

almost_clean_df = remove_outliers(new_dataframe)
final_df = total_clean(almost_clean_df)
print(final_df.head())
final_df.to_csv('final_df.csv', index=False)

   ervaring  500_split  2k tijd binary_trainingtype binary_geslacht  \
0         1      104.6    379.9                   0               0   
1         1      104.7    379.9                   0               0   
2         1      104.3    379.9                   0               0   
3         1      104.0    379.9                   0               0   
4         1      104.1    379.9                   0               0   

  binary_gewichtsklasse  
0                     1  
1                     1  
2                     1  
3                     1  
4                     1  


In [126]:
from sklearn.preprocessing import OneHotEncoder

# List the columns you want to drop
columns_to_drop = ['naam', 'trainingtype', '2k datum', 'binary_geslacht', 'binary_gewichtsklasse']

# Drop the existing columns
almost_clean_df = almost_clean_df.drop(columns=columns_to_drop)

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Columns to encode
columns_to_encode = ['geslacht', 'gewichtsklasse', 'ervaring', 'binary_trainingtype']

# Apply one-hot encoding
encoded_data = encoder.fit_transform(almost_clean_df[columns_to_encode])
encoded_columns = encoder.get_feature_names_out(columns_to_encode)

# Create DataFrame for the encoded columns
encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)

# Drop the columns that were encoded from almost_clean_df
almost_clean_df = almost_clean_df.drop(columns=columns_to_encode)

# Concatenate the one-hot encoded columns with the remaining data
encoded_df = pd.concat([almost_clean_df, encoded_df], axis=1)

# Print the resulting encoded dataframe
print(encoded_df.head())
encoded_df.to_csv('encoded_df.csv', index=False)

   500_split  2k tijd  geslacht_M  geslacht_V  gewichtsklasse_L  \
0      104.6    379.9         1.0         0.0               0.0   
1      104.7    379.9         1.0         0.0               0.0   
2      104.3    379.9         1.0         0.0               0.0   
3      104.0    379.9         1.0         0.0               0.0   
4      104.1    379.9         1.0         0.0               0.0   

   gewichtsklasse_Z  ervaring_0  ervaring_1  binary_trainingtype_0  \
0               1.0         0.0         1.0                    1.0   
1               1.0         0.0         1.0                    1.0   
2               1.0         0.0         1.0                    1.0   
3               1.0         0.0         1.0                    1.0   
4               1.0         0.0         1.0                    1.0   

   binary_trainingtype_1  ...  binary_trainingtype_1101  \
0                    0.0  ...                       0.0   
1                    0.0  ...                       0.0   

In [127]:
# ### per persoon per training de gemiddelde 500_split berekenen

# def gemid_split(data):
#     data['year'] = pd.to_datetime(data['2k datum']).dt.year
    
#     data['unique_naam'] = data['naam'] + '_' + data['year'].astype(str)

#     gemid_data = (data.groupby(['unique_naam', 'binary_trainingtype'], as_index=False).agg({'ervaring': 'first', 'geslacht': 'first', 'gewichtsklasse': 'first', '500_split': 'mean', '2k tijd': 'first', 'binary_geslacht': 'first', 'binary_gewichtsklasse': 'first'}))

#     return gemid_data

In [128]:
# def gemid_total(data):
#     gemid_pp = (data.groupby(['unique_naam'], as_index=False).agg({'ervaring': 'first', '500_split': 'mean', '2k tijd': 'first', 'binary_geslacht': 'first', 'binary_gewichtsklasse': 'first'}))
#     return gemid_pp

# final_gemid_df = gemid_total(final_df)
# print(final_gemid_df)
# final_gemid_df.to_csv('final_gemid_df.csv', index=False)

In [129]:
# ### een dictionary maken van alle gegevens per persoon; dit is meer voor ons overzicht en niet nodig voor het model

# import json

# def dict_per_naam(data):
#   naam_dict = {}

#   for i in range(len(data)):
#       row = data.iloc[i].to_dict()

#       for key, value in row.items():
#           if isinstance(value, pd.Timestamp):
#               row[key] = value.strftime('%Y-%m-%d')

#       name = row.pop('unique_naam')
#       constants = {key: row.pop(key) for key in ['ervaring', 'geslacht', 'gewichtsklasse', '2k tijd']}

#       if name not in naam_dict:
#           naam_dict[name] = {'gegevens': constants, 'tijden': []}

#       naam_dict[name]['tijden'].append(row)

#   print(json.dumps(naam_dict, indent=4, sort_keys=False))

#   return naam_dict

# new_namen_dict = dict_per_naam(new_dataframe)
# gemid_new_namen_dict = dict_per_naam(final_df)
# print(len(gemid_new_namen_dict))

In [130]:
# from sklearn.model_selection import train_test_split

# # Data inladen
# df = groot_gemid_df

# # Eerst de data opschudden om bias te voorkomen
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Zorg ervoor dat elke ervaring, geslacht en gewichtsklasse in elke set vertegenwoordigd zijn
# train_data = pd.DataFrame()
# val_data = pd.DataFrame()
# test_data = pd.DataFrame()

# for ervaring in df['ervaring'].unique():
#     for geslacht in df['geslacht'].unique():
#         for gewichtsklasse in df['gewichtsklasse'].unique():
#             subset = df[(df['ervaring'] == ervaring) & (df['geslacht'] == geslacht) & (df['gewichtsklasse'] == gewichtsklasse)]
#             if not subset.empty:
#                 temp_train, temp_temp = train_test_split(subset, test_size=0.3, random_state=42)
#                 temp_val, temp_test = train_test_split(temp_temp, test_size=0.5, random_state=42)
#                 train_data = pd.concat([train_data, temp_train])
#                 val_data = pd.concat([val_data, temp_val])
#                 test_data = pd.concat([test_data, temp_test])

# # Reset indexen
# train_data.reset_index(drop=True, inplace=True)
# val_data.reset_index(drop=True, inplace=True)
# test_data.reset_index(drop=True, inplace=True)

# # Controleren op juiste verdeling
# print(f"Trainingsdata: {len(train_data)} rijen")
# print(f"Validatiedata: {len(val_data)} rijen")
# print(f"Testdata: {len(test_data)} rijen")

# # Optioneel: data opslaan in aparte bestanden
# train_data.to_csv('train_data.csv', index=False)
# val_data.to_csv('val_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)

In [131]:
# alles = tweek_naar_split(dataframe_op_sec)
# print(alles)
# print(len(alles))

# min_value = alles.min()
# filtered_alles = alles[alles!= min_value]

# unique_values = alles.unique()
# unique_values.sort()

# first_min = unique_values[0]
# second_min = unique_values[1]
# third_min = unique_values[2]
# fourth_min = unique_values[3]
# eighth_min = unique_values[7]

# print(first_min)
# print(second_min)
# print(third_min)
# print(fourth_min)
# print(eighth_min)

# unique_values_desc = alles.unique()
# unique_values_desc.sort()
# unique_values_desc = unique_values_desc[::-1]

# first_largest = unique_values_desc[0]
# second_largest = unique_values_desc[1]
# third_largest = unique_values_desc[2]
# eighth_largest = unique_values_desc[7]

# print(first_largest)
# print(second_largest)
# print(third_largest)
# print(eighth_largest)